### Dependencies

In [1]:
!pip install tqdm
# import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm 
from collections import deque, defaultdict, Counter
# import timeit
# from sklearn.preprocessing import normalize
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.neural_network import MLPClassifier

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Cache Replacement Policy
#### Read here for introduction: https://en.wikipedia.org/wiki/Cache_replacement_policies



#### Blocktraces

In [3]:
train = "cheetah.cs.fiu.edu-110108-113008.1.blkparse"

df = pd.read_csv(train, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace)*0.1)]

len(trainBlockTrace)

132289

### (1) Belady's Optimal Algorithm

In [2]:
def belady_opt(blocktrace, frame):
    '''
    INPUT
    ==========
    blocktrace = list of block request sequence
    frame = size of the cache
    
    OUTPUT
    ==========
    hitrate 
    '''
    infinite_index = 10000 * len(blocktrace) # should be a large integer
    
    block_index = defaultdict(deque) 
    # dictionary with block number as key and list
    # of index value in blocktrace
    
    upcoming_index = defaultdict(int)
    # dictionary with index number as key and value as block
    
    frequency = defaultdict(int)
    # dictionary of block as key and number
    # of times it's been requested so far
    
    recency = list()
    # list of block in order of their request
    
    Cache = deque()
    # Cache with block
    
    dataset = np.array([]).reshape(0,3*frame+1)
    #columns represents the number of block in cache and 
    #3 is the number of features such as frequency, recency and block number
    #+1 is for label 0-1
    
    hit, miss = 0, 0
    
    # populate the block_index
    for i, block in enumerate(tqdm(blocktrace, \
                              desc="buidling index", leave=False)):
        block_index[block].append(i)
        
    # sequential block requests start
    for i, block in enumerate(tqdm(blocktrace, desc="sequence", leave=False)):
        
        # increament the frequency number for the block
        frequency[block] += 1
        
        # make sure block has the value in block_index dictionary 
        # as current seq_number
        if len(block_index[block]) != 0 and block_index[block][0] == i:
            
            # if yes, remove the first element of block_index[block]
            block_index[block].popleft()
        
        # if block exist in current cache
        if block in Cache:
            
            # increment hit
            hit += 1
            
            # update the recency
            recency.remove(block)
            recency.append(block)
            
            # update upcoming_index
            if i in upcoming_index:
                
                # delete old index
                del upcoming_index[i]
        
                if len(block_index[block]) is not 0:
                    # add new upcoming index
                    upcoming_index[block_index[block][0]] = block
                    # remove index from block_index
                    block_index[block].popleft()
                else:
                    # add a large integer as index
                    upcoming_index[infinite_index] = block
                    # increament large integer
                    infinite_index-=1
           
        # block not in current cache
        else:
            
            # increament miss
            miss += 1
            
            # if cache has no free space
            if len(Cache) == frame:
                
                
                # evict the farthest block in future request from cache
                if len(upcoming_index) != 0:
                    
                    # find the farthest i.e. max_index in upcoming_index
                    max_index = max(upcoming_index)
                    
                    # remove the block with max_index from cache
                    Cache.remove(upcoming_index[max_index])
                    
                    # remove the block with max_index from recency dict
                    recency.remove(upcoming_index[max_index])
                    
                    # remove max_index element from upcoming_index
                    del upcoming_index[max_index]
                    
            # add upcoming request of current block in upcoming_index
            if len(block_index[block]) != 0:
                
                # add upcoming index of block
                upcoming_index[block_index[block][0]] = block
               
                # remove the index from block_index 
                block_index[block].popleft()
            
            else:
                
                # add a large integer as index
                upcoming_index[infinite_index] = block
                
                # increament high number
                infinite_index -= 1
                
            
            # add block into Cache
            Cache.append(block)
            
            # add block into recency
            recency.append(block)
            
            
    # calculate hitrate
    hitrate = hit / (hit + miss)

    return hitrate

In [11]:
belady_opt(trainBlockTrace, 100)

0.014498559970972644

### (2) Least Frequently Used 

In [12]:
def LFU(blocktrace, frame):
    '''
    INPUT
    ==========
    blocktrace = list of block request sequence
    frame = size of the cache
    
    OUTPUT
    ==========
    hitrate 
    '''
    
    cache = set()
    
    cache_frequency = defaultdict(int)
    
    frequency = defaultdict(int)
    
    hit, miss = 0, 0
    
    for block in tqdm(blocktrace, leave=False):
        frequency[block] += 1
        
        if block in cache:
            hit += 1
            cache_frequency[block] += 1
        
        elif len(cache) < frame:
            cache.add(block)
            cache_frequency[block] += 1
            miss += 1

        else:
            e, f = min(cache_frequency.items(), key=lambda a: a[1])
            cache_frequency.pop(e)
            cache.remove(e)
            cache.add(block)
            cache_frequency[block] = frequency[block]
            miss += 1
    
    hitrate = hit / ( hit + miss )
    return hitrate

In [13]:
LFU(trainBlockTrace, 100)

0.011059120561800301

### (3) Least Recently Used (LRU)

In [14]:
def LRU(blocktrace, frame):
    '''
    INPUT
    ==========
    blocktrace = list of block request sequence
    frame = size of the cache
    
    OUTPUT
    ==========
    hitrate 
    '''
    
    cache = set()
    
    recency = deque()
    
    hit, miss = 0, 0
    
    for block in tqdm(blocktrace, leave=False):
        
        if block in cache:
            recency.remove(block)
            recency.append(block)
            hit += 1
            
        elif len(cache) < frame:
            cache.add(block)
            recency.append(block)
            miss += 1
            
        else:
            cache.remove(recency[0])
            recency.popleft()
            cache.add(block)
            recency.append(block)
            miss += 1
    
    hitrate = hit / (hit + miss)
    return hitrate

In [15]:
LRU(trainBlockTrace, 100)

0.009766496080550914

### How to implement ML cache replacement policy?

First, we need a base algorithm for our Machine Learning Model to learn. The obvious choice would be, the most desirable and otherwise practically impossible, Belady's Optimal algorithm. This is Supervised Learning approach.

#### How to label data?

There are two approaches for labelling the dataset of bloacktrace. Since the most usable properties of blocks are recency (e.g. in LRU) and frequency (e.g. in LFU), we generate these property by tracing the training data to generate 2 features. In total we have three properties: 1. BlockNumber 2. Recency (of Block request) 3. frequency (of Block request)

Our data would be generated based on Belady's Optimal algorithm. The ML model we desire is predicting which block to evict from cache when miss occures. Therefore, we capture the cache configuration (cache configuration is nothing but an array of cache with elements being blocks themselves) at every time miss occures and label would be 0(for not evicted blocks) and 1(for evicted block). But the problem with this approach is that there is only 1 block with label 1 and remaining blocks are labelled 0. This would make ML model very bias towards 0 and it would never give prediction of label 1. (For example, for cache size 500, 499 block would get label 0 and only one block would have label 1 in dataset for single eviction event. Note that there should be enormous amount of eviction events to be given to ML model to train. Let's say we gave 100 eviction events to ML model, with total 100 (#eviction) x 500 (# cachesize) = 50000 Rows(=datapoints) where 499x100 = 49900 labels are 0 and 1x100 = 100 labels are 1.)

To overcome the bayse error from our dataset, we impliment a interval approach in Belady's Optimal Algorithm. In this approach let's say we have interval of 100 eviction. When first miss occures, we will put x% of blocks into eviction candidates list (which we call buffer cache) and remove the first element that belady's algorithm selects. In next miss, we don't call belady's algorithm for next candidate but remove the second element from Buffer Cache. So we call belady's algorithm only once to get x% block from cache to put aside and consider as potential candidates of eviction, then remove one block at a time when miss occures. Once this buffer cache is empty we call Belady's Optimal algorithm again to get more x% block as next eviction candidates. This approach will ensure x% blocks with label of 1 and (1-x)% of blocks with label of 0.

Next problem is the size of the dataset. Even if we capture the cache configuration with x% of 1s and (1-x)% of 0s everytime miss occures. The size of the dataset could be very large. So we decided to capture the cache configuration with a constant interval of y misses. For example, for a blocktrace of size more than 500k, the interval of 1k can reduce the dataset to feasible size. This reduction would actually helpful as there could be more than one instance of same blockNumber that occures in dataset with different labels(1 and 0) which could lead to ambiguous dataset.

So, apart from cache size, there are paramters such as x(% of eviction candidates) and y(interval of capture the cache configuration) which needs to be optimized.